In [ ]:
# imports
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from yellowbrick.model_selection import LearningCurve
from yellowbrick.classifier import PrecisionRecallCurve
from yellowbrick.classifier import ClassPredictionError
from mlxtend.plotting import plot_learning_curves
from yellowbrick.classifier import ClassificationReport
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import accuracy_score, plot_confusion_matrix, classification_report

# global settings
plt.style.use('classic')
matplotlib.rcParams['figure.figsize'] = (20, 10)

In [ ]:
# evaluation metrics for imbalanced data
def model_evaluation(model, x_train, y_train, x_test, y_test, model_name, y_pred):
    """:arg
    :model, classifier, must have .fit() and .predict()
    :x_train, X, features
    :y_train, y, target
    :x_test, test features
    :y_test, test target
    :model_name, name of the classifier, string
    :y_pred, prediction made by classifier
    """
    # accuracy
    acc = accuracy_score(y_test, y_pred)
    print(f'The accuracy for {model_name}: {acc}')

    # F-measure
    f1score = f1_score(y_test, y_pred, average='micro')
    print(f'F-Measure for {model_name}: {f1score}')

    # Recall
    recallscore = f1_score(y_test, y_pred, average='micro')
    print(f'Recall for {model_name}: {recallscore}')

    # classification report
    visualizer = ClassificationReport(model, support=True, cmap='spectral')

    visualizer.fit(x_train, y_train)        # Fit the visualizer and the model
    visualizer.score(x_test, y_test)        # Evaluate the model on the test data
    visualizer.show()

    # Generate confusion matrix
    matrix = plot_confusion_matrix(model, x_test, y_test, cmap=plt.cm.Blues)
    plt.title(f'Confusion matrix for {model_name} classifier')
    plt.xticks(rotation=65)
    plt.show(matrix)
    plt.show()

    # precision recall curve
    # Create the visualizer, fit, score, and show it
    viz = PrecisionRecallCurve(
        model,
        per_class=True,
        classes=np.unique(y_train), # if LabelEncoder used, use encoder.classes_
        cmap="Set1",
        colors=["purple", "cyan", "slategray", "red", "grey", "yellow", "maroon", "gold", "orange", "olive", "brown", "deeppink", 'darkkhaki', "black", "navy"]
    )
    viz.fit(x_train, y_train)
    viz.score(x_test, y_test)
    viz.show()

    # class prediction error
    # Instantiate the classification model and visualizer
    visualizer = ClassPredictionError(
        model,
        colors=["purple", "cyan", "slategray", "red", "grey", "yellow", "maroon", "gold", "orange", "olive", "brown", "deeppink", 'darkkhaki', "black", "navy"]
    )

    # Fit the training data to the visualizer
    visualizer.fit(x_train, y_train)

    # Evaluate the model on the test data
    visualizer.score(x_test, y_test)

    # Draw visualization
    visualizer.show()

    # Create the learning curve visualizer
    cv = StratifiedKFold(n_splits=10)
    sizes = np.linspace(0.3, 1.0, 10)

    # Instantiate the classification model and visualizer
    visualizer = LearningCurve(
        model, cv=cv, scoring='f1_micro', train_sizes=sizes, n_jobs=4
    )

    visualizer.fit(x_train, y_train)        # Fit the data to the visualizer
    visualizer.show()

    # learning curve with train and test data
    plot_learning_curves(x_train, y_train, x_test, y_test, model, train_marker='o', test_marker='^',
                     scoring='misclassification error', suppress_plot=False, print_model=True, legend_loc='best')


In [ ]:
# sample for RF classifier
model_evaluation(rf, train_tfidf, y_train, test_tfidf, y_test, 'Random Forest', y_pred)